In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time
import pickle

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import dopioe, find_directional_radii, nencioli, psi_params, fit_psi_params


#### New Grid for Nencioli

In [2]:
fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')

res = 1  # 1 km resolution
x_new = np.arange(0, x_grid[-1], res)
y_new = np.arange(0, y_grid[-1], res)
X_new, Y_new = np.meshgrid(x_new, y_new, indexing='ij')
new_points = np.column_stack((X_new.ravel(), Y_new.ravel()))

interp_lon = RegularGridInterpolator((x_grid, y_grid), lon_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)
interp_lat = RegularGridInterpolator((x_grid, y_grid), lat_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)

lon_new = interp_lon(new_points).reshape(len(x_new), len(y_new))
lat_new = interp_lat(new_points).reshape(len(x_new), len(y_new))


#### Interpolate to 1km grid, then apply Nencioli VG method

In [3]:
def interpolate_uv(u, v, X_new, Y_new, angle):
    x_new, y_new = X_new[:,0], Y_new[0,:]

    u_east = np.where(np.abs(u) > 1e30, np.nan, u).astype(float)
    v_north = np.where(np.abs(v) > 1e30, np.nan, v).astype(float)

    u_rot = v_north * np.sin(angle) + u_east * np.cos(angle)
    v_rot = v_north * np.cos(angle) - u_east * np.sin(angle)

    shape_new = X_new.shape
    new_points = np.column_stack((X_new.ravel(), Y_new.ravel()))

    interp_u = RegularGridInterpolator((x_grid, y_grid), u_rot,
                                       method='linear', bounds_error=False, fill_value=np.nan)
    interp_v = RegularGridInterpolator((x_grid, y_grid), v_rot,
                                       method='linear', bounds_error=False, fill_value=np.nan)

    u_new = interp_u(new_points).reshape(shape_new)
    v_new = interp_v(new_points).reshape(shape_new)

    return u_new, v_new


In [4]:
def dopioe_pipeliner(nxc, nyc, cyc, ut, vt, X_new, Y_new, r=30):

    R_grid = np.hypot(nxc - X_new, nyc - Y_new)
    ic, jc = map(int, np.unravel_index(np.argmin(R_grid), R_grid.shape))

    # DOPIOE wont work if too close to boundary
    x_new = X_new[:, 0]
    y_new = Y_new[0, :]
    dx = np.max(np.diff(x_new))  # spacing in x-direction
    dy = np.max(np.diff(y_new))  # spacing in y-direction
    cell_size = np.max([dx, dy])        # average cell size in Euclidean units
    margin = int(np.ceil(r / cell_size)) 

    if (ic < margin or ic >= X_new.shape[0] - margin or
            jc < margin or jc >= X_new.shape[1] - margin):
        return np.nan, np.nan, np.nan, np.array([[np.nan, np.nan],
                                                [np.nan, np.nan]]), np.nan, np.nan, np.nan, np.nan

    # horizontal transect (constant y = y[jc])
    x_mask = np.abs(x_new - nxc) <= r
    x1 = x_new[x_mask]
    y1 = np.full_like(x1, y_new[jc])
    u1 = ut[x_mask, jc]
    v1 = vt[x_mask, jc]
    
    # vertical transect (constant x = x[ic])
    y_mask = np.abs(y_new - nyc) <= r
    y2 = y_new[y_mask]
    x2 = np.full_like(y2, x_new[ic])
    u2 = ut[ic, y_mask]
    v2 = vt[ic, y_mask]
    
    xc, yc, w, Q, _, _, A0 = dopioe(x1, y1, u1, v1, x2, y2, u2, v2)

    cyc_DOPIOE = 'CE' if w < 0 else 'AE'
    
    if (cyc_DOPIOE != cyc) or (np.hypot(nxc - xc, nyc - yc) > 50):
        return np.nan, np.nan, np.nan, np.array([[np.nan, np.nan],
                                                [np.nan, np.nan]]), np.nan, np.nan, np.nan, np.nan
    else:
        w *= 1e-3 # to s^-1

        radii = find_directional_radii(ut, vt, X_new, Y_new, xc, yc, Q)
        R = np.mean([radii['up'], radii['right'], radii['down'], radii['left']])

        q11, q12, q22 = Q[0,0], Q[0,1], Q[1,1]
        dx, dy = X_new - xc, Y_new - yc
        rho2 = q11*dx**2 + 2*q12*dx*dy + q22*dy**2
        rho_search = np.sqrt(np.where(rho2 < 0, np.nan, rho2))
        
        mask_outer = rho_search < max(min(R*1.75, 200), 30) 
        axi, ayi, aui, avi = X_new[mask_outer], Y_new[mask_outer], ut[mask_outer], vt[mask_outer]
        
        df = psi_params(xc, yc, Q, axi, ayi, aui, avi)
        Rc, psi0, A = fit_psi_params(df.rho2, df.Qr, df.vt, A0=A0, Rc_max=200)

        if np.sign(A) != np.sign(w):
            Rc, psi0, A = np.nan, np.nan, A0
            return xc, yc, w, Q, Rc, psi0, A, R
        
    return xc, yc, w, Q, Rc, psi0, A, R


In [ ]:
def build_nenc_dataframe(day, X_new, Y_new, lon_new, lat_new, angle):
    
    rows = []

    fnumber = 1461 + ((day - 1462) // 30)*30
    fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_{fnumber:05}.nc'
    dataset = nc.Dataset(fname)
    u_east = np.transpose(dataset['u_eastward'][:].data, axes=(3, 2, 1, 0))[:, :, -1, :].squeeze()
    v_north = np.transpose(dataset['v_northward'][:].data, axes=(3, 2, 1, 0))[:, :, -1, :].squeeze()
    ocean_time = dataset.variables['ocean_time'][:].data / 86400
    t = np.where(day==ocean_time)[0][0]
    if t.size != 0:
        u, v = u_east[:, :, t], v_north[:, :, t]
        u0, v0 = interpolate_uv(u, v, X_new, Y_new, angle)

        neddy = nencioli(u0.T, v0.T, X_new.T, Y_new.T, 4, 3)[2]
        # Sort so that the highest second-column value comes first
        neddy = neddy[neddy[:, 1].argsort()[::-1]]
        
        for idx, (nxc0, nyc0, cyc_indicator) in enumerate(neddy):
            cyc = 'CE' if cyc_indicator == 1 else 'AE'
            nic_idx, njc_idx = np.where((X_new == nxc0) & (Y_new == nyc0))
            if nic_idx.size:
                nic0, njc0 = nic_idx[0], njc_idx[0]
            else:
                nic0, njc0 = np.nan, np.nan

            xc, yc, w, Q, Rc, psi0, A, R = dopioe_pipeliner(nxc0, nyc0, cyc,
                                                          u0, v0, X_new, Y_new, r=30)
            
            rows.append({
                'Eddy_idx': idx,
                'Day': day,
                'Cyc': cyc,
                'nLon': lon_new[nic0, njc0],
                'nLat': lat_new[nic0, njc0],
                'nxc': nxc0,
                'nyc': nyc0,
                'nic': nic0,
                'njc': njc0,
                'xc': xc,
                'yc': yc,
                'w': w,
                'q11': Q[0,0],
                'q12': Q[1,0],
                'q22': Q[1,1],
                'Rc': Rc,
                'psi0': psi0,
                'A': A,
                'R': R
            })

    df_data_day = pd.DataFrame(rows)
    return df_data_day

start_day = 4510 # 1462
num_days = 6140 # last valid day 10650 # 9188 num of days then

dfs = []
for day in range(start_day, start_day + num_days + 1):
    dfs.append(build_nenc_dataframe(day, X_new, Y_new, lon_new, lat_new, angle))
    df_data = pd.concat(dfs, ignore_index=True)
    df_data.to_pickle(f"/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/df_data_{start_day}_{start_day+num_days}.pkl")
    if day % 20 == 0:
        print(day)
        